In [ ]:
import geopandas as gpd
import rioxarray
import rasterstats as rs
import numpy as np
import xarray as xr

### Exploring Land Surface Temperature

ESA Land Surface Temperature Climate Change Initiative (LST_cci): Monthly Multisensor Infra-Red (IR) Low Earth Orbit (LEO) land surface temperature (LST) time series level 3 supercollated (L3S) global product (1995-2020), version 2.00. More information from the CEDA Archive can be found [here](https://catalogue.ceda.ac.uk/uuid/785ef9d3965442669bff899540747e28).

Now let's explore Land Surface Temperature, but this time let's try to access directly from the URL

In [ ]:
# url = ("https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc?download=1")

# original link above doesn't work, must add `#mode=bytes` to the end (see: https://github.com/Unidata/netcdf4-python/issues/1043)
url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc#mode=bytes"

In [ ]:
ds_disk = xr.open_dataset(url)
ds_disk

In [ ]:
# # Open the world-wide NetCDF
# ds = rioxarray.open_rasterio(url)
# ds = ds.rio.write_crs("epsg:4326")

Before we do any processing of the data, we will want to clip the global dataset down to our AOI (continent of Africa) to reduce the size and processing time. To do this, we'll import the national administrative boundaries (admin 0) for Africa in order to create a bounding box for the continent.

In [ ]:
import geopandas as gpd

admin0_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
# admin0_gdf

print(admin0_gdf.crs)

In [ ]:
xmin, ymin, xmax, ymax = admin0_gdf.geometry.total_bounds
print(xmin, ymin, xmax, ymax)

In [ ]:
# Subset to bounding box of African continent
ds_disk.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
lst_clip = ds_disk["lst"].rio.clip_box(
    minx=-25.35874748,
    miny=-46.9813795,
    maxx=63.5026492,
    maxy=37.560954,
    crs="EPSG:4326",
)

In [ ]:
lst_clip

In [ ]:
lst_clip.coords

In [ ]:
# Convert from Kelvin to Celsius
lst_africa_c = lst_clip - 273.15
lst_africa_c

# does this work correctly? It appears to, but bounds still fairly high in some pleaces

We'll visualize only 1 time step here - but it would be great to visualize the whole time series:
* we could visualize instead as a line graph
* or as visualization that loops over the entire time period 

But more importantly, we're going to have to aggregate these daily values to monthly. 

In [ ]:
lst_africa_c.dims
lst_africa_c.coords

In [ ]:
# # selecting single time point (first time-step)
# lst_2d = lst_africa_c.isel(time=0)
# lst_2d

In [ ]:
# selecting single time point (first time-step)
lst_2d = lst_africa_c.squeeze()
lst_2d

In [ ]:
lst_2d.dims
lst_2d.coords

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.contourf(lst_2d, 20, cmap=plt.get_cmap("coolwarm"))
plt.colorbar()
plt.show()

# Would prefer hvplot, but datashader not yet supported on python 3.11

In [ ]:
admin2_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_2&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
admin2_gdf
print(admin2_gdf.crs)

In [ ]:
import pandas as pd
from rasterstats import zonal_stats

In [ ]:
admin2_gdf.crs

In [ ]:
admin2_lst = rs.zonal_stats(
    admin2_gdf,
    lst_2d.values,
    affine=lst_2d.rio.transform(),
    nodata=lst_2d.rio.nodata,
    stats="mean",
)
admin2_lst

In [ ]:
# admin2_lst = pd.DataFrame(
#     zonal_stats(
#         admin2_gdf,
#         lst_2d.values,
#         affine=lst_2d.rio.transform(),
#         nodata=lst_2d.rio.nodata,
#     ),
#     index=admin2_gdf.index,
# )
# admin2_lst